In [ ]:
import pandas as pd
import numpy as np

import bokeh.plotting
import bokeh.io

from bokeh.models import IndexFilter
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)


bokeh.io.output_notebook()

Loading BokehJS ...

In [5]:
# Load in the metadata
ferm_locations = pd.read_csv("data/ferm.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

In [6]:
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

In [7]:
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

In [8]:
# Options for x- and y- selector; omit part. num., gender, and insomnia
class_options = list(ferm_locations_final.Class.unique())

class_selector = bokeh.models.Select(
    title="class", options=class_options, value="Dairy", width=200,
)

In [9]:
render_cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ["#1f77b3"] * len(ferm_locations_final),
    }
)

In [10]:
cds = bokeh.models.ColumnDataSource(ferm_locations_final)


In [11]:
p = bokeh.plotting.figure(
    frame_height=400,
    frame_width=700,
    x_axis_label="Lat",
    y_axis_label="Long",
    tools =['hover', 'wheel_zoom', 'pan', 'reset'],
    x_axis_type = 'mercator',
    y_axis_type = 'mercator'
)
p.add_tile(tile_provider)


circle = p.circle(
    source=render_cds, x="x", y="y", fill_color="color", line_color="color",
)

In [12]:
filter1 = IndexFilter(indices=[])

jscode = """
// Extract what we want to color by from selector
var class = class_selector.value;

const indices = []
for (var i = 0; i < cds.get_length(); i++) {
if (cds.data['Class'][i] == type) {
  indices.push(i)
}
}

filter1.indices = indices

"""

args = dict(
    cds=cds,
    class_selector=class_selector,
)

class_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

In [13]:
# jscode = """
# // Extract what we want to color by from selector
# var class = class_selector.value;


# // For convenience, get views into the ColumnDataSource data
# var x = cds.data['x'];
# var y = cds.data['y'];

# // Loop through and update
# var xLen = x.length;
# for (var i = 0; i < xLen; i++) {
#     y[i] = normalPDF(x[i], mu, sigma);
# }

# for (var i = 0; i < n; i++) {
#     if (cds.data['Class'][i]==class) {
#         colors[i] = '#1f77b3';
#     }
# }

# cds.change.emit();
# """

# args = dict(
#     cds=cds,
#     class_selector=class_selector,
# )

In [14]:
layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        class_selector,
    ),
)

bokeh.io.show(layout)

In [15]:
bokeh.io.save(layout, 'bokeh_map.html')

/opt/miniconda3/envs/basics/lib/python3.9/site-packages/bokeh/io/saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/miniconda3/envs/basics/lib/python3.9/site-packages/bokeh/io/saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/lianamerk/git/fermap/bokeh_map.html'

In [19]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show, row
from bokeh.models import ColumnDataSource, CustomJS, MultiChoice

df = pd.DataFrame()
df['name'] = ['a','a','a','b','b','b','c','c','c']
df['sample'] =np.random.randint(1,3,9)
df['x'] = np.random.randint(0,10,9)
df['y'] = np.random.randint(0,10,9)

plot = figure(width=400, height=400)


name_dict={'name':[],'legend':[],'label':[]}
for name in df.name.unique():  
    source = ColumnDataSource(df[df['name']==name].groupby(['sample']).median())
    name_glyph = plot.circle('x', 'y', source=source, legend_label=name)
    name_dict['name'].append(name_glyph)
    name_dict['label'].append(name)

name_legend_dict={'name':[]}
for label in range(len(df.name.unique())):
    name_dict['legend'].append(plot.legend.items[label])
    
# Set up MultiChoice widget
initial_value = [df.name[0]]
options = list(df.name.unique())

multi_choice = MultiChoice(value=initial_value, options=options, max_items=3, title='Selection:')

for i in range(len(options)):
    if name_dict['label'][i] in initial_value:
        name_dict['name'][i].visible = True;
        name_dict['legend'][i].visible = True;
    else:
        name_dict['name'][i].visible = False;
        name_dict['legend'][i].visible = False;   
    

callback = CustomJS(args=dict(name_dict=name_dict, multi_choice=multi_choice), code="""
var selected_vals = multi_choice.value;
var index_check = [];

for (var i = 0; i < name_dict['name'].length; i++) {
    index_check[i]=selected_vals.indexOf(name_dict['label'][i]);
        if ((index_check[i])>= 0) {
            name_dict['name'][i].visible = true;
            name_dict['legend'][i].visible = true;
            }
        else {
            name_dict['name'][i].visible = false;
            name_dict['legend'][i].visible = false;
        }
        
    }
""")

multi_choice.js_on_change('value', callback)

plot.legend.background_fill_alpha = 0.25
layout = row(plot,multi_choice)
show(layout)

In [21]:
bokeh.io.save(layout, "multiselect.html")

'/Users/lianamerk/git/fermap/multiselect.html'